In [59]:
# T2. Exercise / 예시문제 : 백화점고객의 1년간 데이터 (dataq 공식 예제)
# 풀이

import pandas as pd

X_train = pd.read_csv('./data/Exercise/X_train.csv', encoding='euc-kr')
X_test = pd.read_csv('./data/Exercise/X_test.csv', encoding='euc-kr')
y_train = pd.read_csv('./data/Exercise/y_train.csv', encoding='euc-kr')

# print(X_train.head())
# print(y_train.head())
# print(X_train.isnull().sum()) # 환불금액(2295)
# print(X_test.isnull().sum()) # 환불금액(1611)
# print(X_train.shape, X_test.shape, y_train.shape) # (3500, 10) (2482, 10) (3500, 2)

# print(X_train.describe(include='object')) # 주구매상품 42
# print(X_test.describe(include='object')) # 주구매상품 41

# train, test 합치고 (id 뽑고)
# 전처리 환불금액 0으로
# 라벨 인코딩
# 스케일링 minmax
# 다시 train, test로 나누고
# train_test_split : tr, val
# 모델링 gender(분류) : DecisionTree, RandomForest, xgboost, svm, KNeighbors
# roc_auc_score 평가
# submit

X_train_id = X_train.pop('cust_id')
X_test_id = X_test.pop('cust_id')

df = pd.concat([X_train, X_test])
# print(df)
df['환불금액'] = df['환불금액'].fillna(0)
# print(df.isnull().sum())
# print(df.describe(include='object'))

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
cols = df.describe(include='object').columns
le = LabelEncoder()
for col in cols:
    df[col] = le.fit_transform(df[col])
# print(df.info())

# 스케일링 minmax
from sklearn.preprocessing import minmax_scale

# print(df.describe())
cols = df.columns
for col in cols:
    df[col] = minmax_scale(df[col])
# print(df.describe())

X_train = df[:X_train.shape[0]]
X_test = df[X_train.shape[0]:]
# print(X_train.shape, X_test.shape)

# train_test_split : tr, val
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2022)
# print(help(train_test_split))
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

# 모델링 gender(분류) : DecisionTree, RandomForest, xgboost, svm, KNeighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
# import xgboost
# print(dir(xgboost))
from sklearn.svm import SVC
# import sklearn.svm
# print(dir(sklearn.svm))
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

model = DecisionTreeClassifier(random_state=2022)
model.fit(X_tr, y_tr['gender'])
# print(X_val.head())
# print(y_val['gender'].head())
# print(y_val['gender'].value_counts())
print(model.score(X_val, y_val['gender']))

# model.fit(X_tr, y_tr)
# pred = model.predict(X_val)
# # print(y_val)
# # print(pred)
# print(roc_auc_score(y_val['gender'], pred[:,1]))

model = RandomForestClassifier(random_state=2022)
model.fit(X_tr, y_tr['gender'])
print(model.score(X_val, y_val['gender']))

# model.fit(X_tr, y_tr)
# print(model.score(X_val, y_val['gender']))
# model.fit(X_tr, y_tr)
# pred = model.predict(X_val)
# print(roc_auc_score(y_val['gender'], pred[:,1]))

model = XGBClassifier(random_state=2022)
model.fit(X_tr, y_tr['gender'])
print(model.score(X_val, y_val['gender']))

model = SVC(random_state=2022)
model.fit(X_tr, y_tr['gender'])
print(model.score(X_val, y_val['gender']))

model = KNeighborsClassifier()
model.fit(X_tr, y_tr['gender'])
print(model.score(X_val, y_val['gender']))


model = RandomForestClassifier(random_state=2022)
model.fit(X_train, y_train['gender'])
pred = model.predict(X_test)

submit = pd.DataFrame()
submit['cust_id'] = X_test_id
submit['gender'] = pred
# print(y_train.head())
submit.to_csv('Exercise_1.csv', index=False)

submit = pd.read_csv('Exercise_1.csv')
print(submit.head())

0.54
0.6171428571428571
0.6114285714285714
0.6142857142857143
0.6028571428571429


c:\Users\user\.conda\envs\han\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


   cust_id  gender
0     3500       1
1     3501       0
2     3502       0
3     3503       0
4     3504       0


In [65]:
# 해답

import pandas as pd

X = pd.read_csv('./data/Exercise/X_train.csv', encoding='euc-kr')
y = pd.read_csv('./data/Exercise/y_train.csv', encoding='euc-kr')
test = pd.read_csv('./data/Exercise/X_test.csv', encoding='euc-kr')

# print(X.shape, y.shape, test.shape)

X = X.fillna(0)
test = test.fillna(0)

X = X.drop(['cust_id'], axis=1)
cust_id = test.pop('cust_id')

from sklearn.preprocessing import LabelEncoder
cols = ['주구매상품', '주구매지점']
for col in cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    test[col] = le.transform(test[col])

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=2022, max_depth=5)
model.fit(X, y['gender'])
print(model.score(X, y['gender']))

pred = model.predict_proba(test)

output = pd.DataFrame()
output['cust_id'] = cust_id
output['gender'] = pred[:,1]
output.to_csv('Exercise_2.csv', index=False)
print(pd.read_csv('Exercise_2.csv').head())

0.6874285714285714
   cust_id    gender
0     3500  0.435672
1     3501  0.197256
2     3502  0.177326
3     3503  0.420662
4     3504  0.484252
